In [23]:
# pip install python-dotenv
# pip install psycopg2-binary

In [ ]:
import os
import requests
from dotenv import load_dotenv
import psycopg2
from bs4 import BeautifulSoup
import re
import time

#env 파일 환경변수 로드
load_dotenv() 

# KaKao API 설정
KAKAO_API_KEY = os.getenv("KAKAO_REST_API_KEY")
headers = { "Authorization": f"KakaoAK {KAKAO_API_KEY}" }
url = "https://dapi.kakao.com/v2/local/search/keyword.json"

In [ ]:
# PostgreSQL 연결 설정
try:
    conn = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )
    cur = conn.cursor()
    print("DB connect 성공")
except Exception as e:
    print("실패",e)

DB connect 성공


In [ ]:
def save_cafe_db(cafe):
    
    cur.execute("""
        INSERT INTO cafe (kakao_id, name, address, road_address, phone,
        category, longitude, latitude, place_url)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (kakao_id) DO NOTHING;
                """,(
                    cafe['id'], cafe['place_name'], cafe['address_name'],cafe['road_address_name'],
                    cafe['phone'], cafe['category_group_name'], float(cafe['x']), float(cafe['y']),
                    cafe['place_url']
                ))
    conn.commit()
    
def fetch_all_cafe(query="강남 카페"):
    all_cafes = []
    for page in range(1,4):
        params={
            "query": query,
            "size":15,
            "page":page
            }
        res = requests.get(url,headers=headers,params=params)
        data = res.json()
    
        cafes = data['documents']
        if not cafes:
            break
        all_cafes.extend(cafes)
    return all_cafes
    
    
# 실행

cafe_region = ["강남 카페","홍대 카페","신촌 카페","이대 카페","이태원 카페","성수 카페","연남동 카페",\
    "한남동 카페","압구정 카페","청담 카페","종로 카페","인사동 카페","건대입구 카페"]

for cafe_r in cafe_region:
    cafes = fetch_all_cafe(cafe_r)
    for cafe in cafes:
        save_cafe_db(cafe)
    
    print(f"저장 완료{len(cafes)}개 수집됨")
    
# 정리
cur.close()
conn.close()

DB connect 성공
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨
저장 완료45개 수집됨


In [ ]:
def crawl_place_info(place_url):
    headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    try:
        res = requests.get(place_url,headers=headers, timeout=5)
        if res.status_code != 200:
            print("요청 실패",res.status_code)
            return None
    
        soup = BeautifulSoup(res.text,"html.parser")
        blog_tag = soup.find("a",class_="link_blog")

        if blog_tag:
            text=blog_tag.get_text(strip=True)
            match = re.search(r'블로그 리뷰\s*(\d+)',text)
            if match:
                return int(match.group(1))
        return 0

    except Exception as e:
        print(" 크롤링 오류", e )
        
cur.execute("SELECT id, place_url FROM cafe")
rows = cur.fetchall()
    
for cafe_id, url in rows:
    count = crawl_place_info(url)
    print(f"[{cafe_id}] 블로그 리뷰 수:{count}")
    timesleep(1)

[1] 블로그 리뷰 수:0
[2] 블로그 리뷰 수:0
[3] 블로그 리뷰 수:0
[4] 블로그 리뷰 수:0
[5] 블로그 리뷰 수:0
[6] 블로그 리뷰 수:0
[7] 블로그 리뷰 수:0
[8] 블로그 리뷰 수:0
[9] 블로그 리뷰 수:0
[10] 블로그 리뷰 수:0
[11] 블로그 리뷰 수:0
[12] 블로그 리뷰 수:0
[13] 블로그 리뷰 수:0
[14] 블로그 리뷰 수:0
[15] 블로그 리뷰 수:0
[16] 블로그 리뷰 수:0
[17] 블로그 리뷰 수:0
[18] 블로그 리뷰 수:0
[19] 블로그 리뷰 수:0
[20] 블로그 리뷰 수:0
[21] 블로그 리뷰 수:0
[22] 블로그 리뷰 수:0
[23] 블로그 리뷰 수:0
[24] 블로그 리뷰 수:0
[25] 블로그 리뷰 수:0
[26] 블로그 리뷰 수:0
[27] 블로그 리뷰 수:0
[28] 블로그 리뷰 수:0
[29] 블로그 리뷰 수:0
[30] 블로그 리뷰 수:0
[31] 블로그 리뷰 수:0
[32] 블로그 리뷰 수:0
[33] 블로그 리뷰 수:0
[34] 블로그 리뷰 수:0
[35] 블로그 리뷰 수:0
[36] 블로그 리뷰 수:0
[37] 블로그 리뷰 수:0
[38] 블로그 리뷰 수:0
[39] 블로그 리뷰 수:0
[40] 블로그 리뷰 수:0
[41] 블로그 리뷰 수:0
[42] 블로그 리뷰 수:0
[43] 블로그 리뷰 수:0
[44] 블로그 리뷰 수:0
[45] 블로그 리뷰 수:0
[136] 블로그 리뷰 수:0
[137] 블로그 리뷰 수:0
[138] 블로그 리뷰 수:0
[139] 블로그 리뷰 수:0
[140] 블로그 리뷰 수:0
[141] 블로그 리뷰 수:0
[142] 블로그 리뷰 수:0
[143] 블로그 리뷰 수:0
[144] 블로그 리뷰 수:0
[145] 블로그 리뷰 수:0
[146] 블로그 리뷰 수:0
[147] 블로그 리뷰 수:0
[148] 블로그 리뷰 수:0
[149] 블로그 리뷰 수:0
[150] 블로그 리뷰 수:0
[151] 블로그 리뷰 수:0
[152] 블로그 리뷰 수:0


KeyboardInterrupt: 